# First improvement attempts:

### <font color='blue'> Investigate improvements to feature selection / engineering </font>

* <font color='gray'> _statistical & data engineering considerations:_ </font>
    <br><font color='gray'> 1. comparison between scaled and unscaled features</font>
    <br><font color='gray'> 2. elimination of highly cross-correlated variables</font>
    <br><font color='gray'> 3. variable exclusion through ANOVA significance tests</font> <br>
* <font color='gray'> _scientific (e.g. chemistry) considerations:_ </font>
    <br><font color='gray'> 4. use of the VSA metrics: presence vs. absence </font>
    <br><font color='gray'> 5. type of the VSA metric: surface area vs. percent of surface area</font>
    <br><font color='gray'> 6. reducing features to retain only widely-accepted factors affecting drug bioavailability</font>

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('BApredsV1').getOrCreate()

# load the data
data = spark.read.csv("bioavailability_data_final.csv",inferSchema=True,sep=',',header=True)

* first, let's start by splitting the features into each VSA type:  **quantity VSA** _vs._ **percentage of VSA**

In [ ]:
'''
# FEATURE SET 1a/1b: compare two variants of VSA calculation
'''
# set1a features property_VSA ranges each in absolute quantity VSA
feature_set1a = ['MolWt','ExactMolWt','qed','MolLogP','MolMR','VSA_total','LabuteASA','TPSA',
                 'MaxPartialCharge','MinPartialCharge','MaxAbsPartialCharge','MinAbsPartialCharge',
                 'NumHAcceptors','NumHDonors','HeavyAtomCount','NumHeteroatoms','NumRotatableBonds',
                 'NHOHCount','NOCount','FractionCSP3','RingCount','NumAliphaticRings','NumAromaticRings',
                 'NumAliphaticHeterocycles','NumAromaticHeterocycles','NumSaturatedHeterocycles',
                 'NumSaturatedRings','BalabanJ','BertzCT','HallKierAlpha',
                 'PEOE_VSA1','PEOE_VSA2','PEOE_VSA3','PEOE_VSA4','PEOE_VSA5','PEOE_VSA6','PEOE_VSA7',
                 'PEOE_VSA8','PEOE_VSA9','PEOE_VSA10','PEOE_VSA11','PEOE_VSA12','PEOE_VSA13','PEOE_VSA14',
                 'SMR_VSA1','SMR_VSA2','SMR_VSA3','SMR_VSA4','SMR_VSA5','SMR_VSA6','SMR_VSA7','SMR_VSA8',
                 'SMR_VSA9','SMR_VSA10','SlogP_VSA1','SlogP_VSA2','SlogP_VSA3','SlogP_VSA4','SlogP_VSA5',
                 'SlogP_VSA6','SlogP_VSA7','SlogP_VSA8','SlogP_VSA9','SlogP_VSA10','SlogP_VSA11','SlogP_VSA12']

# set1a features property_VSA ranges each in relative quantity (percent) of total VSA
feature_set1b = ['MolWt','ExactMolWt','qed','MolLogP','MolMR','VSA_total','LabuteASA','TPSA',
                   'MaxPartialCharge','MinPartialCharge','MaxAbsPartialCharge','MinAbsPartialCharge',
                   'NumHAcceptors','NumHDonors','HeavyAtomCount','NumHeteroatoms','NumRotatableBonds',
                   'NHOHCount','NOCount','FractionCSP3','RingCount','NumAliphaticRings','NumAromaticRings',
                   'NumAliphaticHeterocycles','NumAromaticHeterocycles','NumSaturatedHeterocycles',
                   'NumSaturatedRings','BalabanJ','BertzCT','HallKierAlpha',
                   'fracVSA_PEOE01','fracVSA_PEOE02','fracVSA_PEOE03','fracVSA_PEOE04','fracVSA_PEOE05',
                   'fracVSA_PEOE06','fracVSA_PEOE07','fracVSA_PEOE08','fracVSA_PEOE09','fracVSA_PEOE10',
                   'fracVSA_PEOE11','fracVSA_PEOE12','fracVSA_PEOE13','fracVSA_PEOE14',
                   'fracVSA_SMR01','fracVSA_SMR02','fracVSA_SMR03','fracVSA_SMR04','fracVSA_SMR05',
                   'fracVSA_SMR06','fracVSA_SMR07','fracVSA_SMR08','fracVSA_SMR09','fracVSA_SMR10',
                   'fracVSA_SlogP01','fracVSA_SlogP02','fracVSA_SlogP03','fracVSA_SlogP04',
                   'fracVSA_SlogP05','fracVSA_SlogP06','fracVSA_SlogP07','fracVSA_SlogP08',
                   'fracVSA_SlogP09','fracVSA_SlogP10','fracVSA_SlogP11','fracVSA_SlogP12']

* next, let's eliminate highly cross-correlated variables

In [ ]:
'''
# FEATURE SET 2a/2b: copy feature set 1a/1b and remove highly [e.g. 95%] cross-correlated vars
'''
import numpy as np
import pandas as pd

# Set 2a is derived from Set 1a
feature_set2a = []
df = data.select(feature_set1a).toPandas()
corr_matrix = df.corr().abs() # builds correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool)) # top-right triangle of matrix
cols_to_drop = [column for column in upper.columns if any(upper[column] > 0.95)] # isolate features w/ corr > 0.95
for item in feature_set1a:
    if item in cols_to_drop: 
        pass
    else:
        feature_set2a.append(item) 
        
# Set 2b is derived from Set 1b
feature_set2b = []
df = data.select(feature_set1b).toPandas()
corr_matrix = df.corr().abs() # builds correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool)) # top-right triangle of matrix
cols_to_drop = [column for column in upper.columns if any(upper[column] > 0.95)] # isolate features w/ corr > 0.95
for item in feature_set1b:
    if item in cols_to_drop: 
        pass
    else:
        feature_set2b.append(item) 


* check the effect of entirely skipping any VSA calculations

In [ ]:
# FEATURE SELECTION (3): feature set 2, minus VSA/fracVSA related calculations
feature_set3 = ['MolWt', 'qed', 'MolLogP', 'TPSA', 
                'MaxPartialCharge', 'MinPartialCharge', 
                'NumHAcceptors', 'NumHDonors', 'NumHeteroatoms', 
                 'NumRotatableBonds', 'FractionCSP3', 
                'RingCount', 'NumAliphaticRings', 'NumAromaticRings',
                 'NumAliphaticHeterocycles', 'NumAromaticHeterocycles', 
                 'NumSaturatedHeterocycles', 'NumSaturatedRings', 
                'BalabanJ', 'BertzCT', 'HallKierAlpha']

* check the effect if we keep **only** metrics widely recognized to influence bioavailability

In [ ]:
''' 
# FEATURE SETS 4a/4b: Keep only the criteria relevant for Lipinski, Ghose, Veber, REOS (etc.) filters/rules.  
'''
# feature set 4a uses NHOH / NO counts instead of H donor/acceptor 
feature_set4a = ['MolWt', 
                 'MolLogP',
                 'MolMR',
                 'TPSA',
                 'NHOHCount','NOCount',
                 'FractionCSP3',
                 'NumRotatableBonds',
                 'HeavyAtomCount']

# feature set 4b uses H donor/acceptor counts instead of NHOH/NO counts              
feature_set4b = ['MolWt', 
                 'MolLogP',
                 'MolMR',
                 'TPSA',
                 'NumHAcceptors','NumHDonors',
                 'FractionCSP3',
                 'NumRotatableBonds',
                 'HeavyAtomCount']

* ANOVA analysis

In [ ]:
'''# ANOVA between Low / Medium / High BA ... versus features
'''
# calculate statistic
import scipy.stats
from scipy.stats import f_oneway

# select data subset w/ drug name, BA_pct, and feature set 1b 
cols_temp = ['Name','BA_pct']
cols_temp.extend(list(feature_set1b))
df_temp1 = data.select(cols_temp)

# ANOVA will need a categorical label column, so here we group BA_pct into 3 buckets
from pyspark.ml.feature import Bucketizer
bucketizer = Bucketizer(splits=[0, 33.3, 66.7, float('Inf')],inputCol="BA_pct", outputCol="label_bin3")
df_out2 = bucketizer.setHandleInvalid("keep").transform(df_temp1)

# select all columns and send to Pandas
cols_temp = ['Name','BA_pct','label_bin3']
cols_temp.extend(list(feature_set1b))
df_out2 = df_out2.select(cols_temp).toPandas()

# assign names to the three groups for ANOVA 
low = df_out2[df_out2['label_bin3']==0]
mid = df_out2[df_out2['label_bin3']==1]
hi = df_out2[df_out2['label_bin3']==2]

#calculate P-Value in each ANOVA group
varlist = list(feature_set1b)
vars_to_keep = []
vars_failed = []
for var_x in varlist:
    f_stat,p_val = f_oneway(low[var_x],mid[var_x],hi[var_x])
    def check(p_val):
        if p_val < 0.05:
            return 'ok'
        else:
            return 'FAIL'
    independence = check(p_val)
    if independence == 'ok':
        vars_to_keep.append(var_x)
    else:
        vars_failed.append(var_x)

# remove variables not passing ANOVA test from feature Sets 1b and 2b
F2bANOVA = list(feature_set2b)
F1bANOVA = list(feature_set1b)
for var_x in vars_failed: 
    F1bANOVA.remove(var_x)
    if var_x in F2bANOVA:
        F2bANOVA.remove(var_x)

* save all feature-set variants as a features catalog, for quick later reference

In [ ]:
featuresCatalog = {}
featuresIndexDict = {}

featnum = ['F1a','F1b','F2a','F2b','F3','F4a','F4b','F1bANOVA','F2bANOVA']

featDescrip = ['F1a: All calculations, with Property-VSA ranges given as quantity SA per Property.',
               'F1b: All calculations, with Property-VSA ranges given as percent SA per Property.',
               'F2a: Same as F1a, but excluding all features having cross correlations of > 0.95.',
               'F2b: Same as F1b, but excluding all features having cross correlations of > 0.95.',
               'F3: Same as F2a/F2b, but excluding Property-VSA ranges.',
               'F4a: Metrics related to common filters (Lipinski, Ghose, Veber, REOS); uses NHOH/NO counts instead of H-donor/H-acceptor counts.',
               'F4a: Metrics related to common filters (Lipinski, Ghose, Veber, REOS); uses H-donor/H-acceptor counts instead of NHOH/NO counts.',
               'F1bANOVA: F1b vars passing ANOVA (0.05 Sig.) across 3 bioavailability groups (low/mid/hi)',
               'F2bANOVA: F2b vars passing ANOVA (0.05 Sig.) across 3 bioavailability groups (low/mid/hi)']

featVariants = [feature_set1a,feature_set1b,
                feature_set2a,feature_set2b,
                feature_set3,
                feature_set4a,feature_set4b,
                F1bANOVA, F2bANOVA]

i = 0
for featList in featVariants:
    featName = featnum[i]
    
    featuresIndexDict[featName] = featList
    
    featuresCatalog[i] = {}
    featuresCatalog[i]['name'] = featName
    featuresCatalog[i]['description'] = featDescrip[i]
    featuresCatalog[i]['featureCount'] = len(featList)
    featuresCatalog[i]['features'] = featList
    
    i += 1

featuresDF = pd.DataFrame.from_dict(featuresCatalog,orient='index',dtype=object)
featuresDF = featuresDF.infer_objects()
featuresDF.dtypes

featuresDF.to_pickle('featuresCatalogDF_2022-08-17_new.pickle')
featuresDF.to_parquet('featuresCatalogDF_2022-08-17_new.parquet')

featuresDF.head(7)